# Import Dependencies

In [1]:
# Import our dependencies
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Load in CSVs

In [2]:
math_file_path = os.path.join("Student_alcohol_consumption", "student-mat.csv")
math = pd.read_csv(math_file_path)

por_file_path = os.path.join("Student_alcohol_consumption", "student-por.csv")
por = pd.read_csv(por_file_path)

# Preprocessing
###### Very little cleaning needed to be done. Only the grades needed to be bucketed into pass and fail. OneHotEncoder and StandardScaler could handle the rest.

In [3]:
# clean bucket the grades
# 10 and above is a pass
# 9 and below is a fail

def encode_grades(data):
    # bucket the grades into passing(1) and failling(0)
    
    # math
    # failling
    data.loc[(data["G1"] < 10), "G1"] = 0
    data.loc[(data["G2"] < 10), "G2"] = 0
    data.loc[(data["G3"] < 10), "G3"] = 0

    #passing
    data.loc[(data["G1"] >= 10), "G1"] = 1
    data.loc[(data["G2"] >= 10), "G2"] = 1
    data.loc[(data["G3"] >= 10), "G3"] = 1
    
    display(data)
    
    return

encode_grades(math)
encode_grades(por)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,0,0,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,0,0,0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,0,0,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,1,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,0,0,0
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,1,1,1
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,1,0,0
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,1,1,1


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,1,1
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,0,1,1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,1,1,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,1,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,1,1,1
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,1,1,1
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,1,1,0
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,1,1,1


# Deep Neural Networks
###### I tried and would of liked to make this look nicer by making a function to handle this so I would not have needed to just copy and paste it 6 times.

In [4]:
def dnn(data, target, drop, file_name):

    # Generate our categorical variable list
    encode_cat = data.dtypes[data.dtypes == "object"].index.tolist()

    # Check the number of unique values in each column
    data[encode_cat].nunique()

    # Create the OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit the encoder and produce encoded DataFrame
    encode_df = pd.DataFrame(enc.fit_transform(data[encode_cat]))

    # Rename encoded columns
    encode_df.columns = enc.get_feature_names(encode_cat)

    # Merge the two DataFrames together and drop the Country column
    data = data.merge(encode_df,left_index=True,right_index=True).drop(encode_cat, 1)

    # Split our preprocessed data into our features and target arrays
    y = data[target].values
    X = data.drop(drop,1).values

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

    # Create a StandardScaler instance
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # Define the model - deep neural net
    number_input_features = len(X_train[0])
    hidden_nodes_layer1 =  len(X_train[0]) * 2
    hidden_nodes_layer2 = len(X_train[0]) * .1

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Train the model
    fit_model = nn.fit(X_train_scaled,y_train,epochs=50, verbose=0)

    # Evaluate the model using the test data
    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test, verbose=0)
    
    # save model
    nn.save(file_name)
    
    return model_accuracy * 100

In [5]:
def target_summary():
    
    G1 = ["G1", "G2", "G3"]
    G2 = ["G2", "G3"]
    G3 = ["G3"]

    summary_table = pd.DataFrame(
        {
            "Math G1": [dnn(math, "G1", G1, "trained_math_G1.h5")],
            "Math G2": [dnn(math, "G2", G2, "trained_math_G2.h5")],
            "Math G3": [dnn(math, "G3", G3, "trained_math_G3.h5")],
            "Portuguese G1": [dnn(por, "G1", G1, "trained_por_G1.h5")],
            "Portuguese G2": [dnn(por, "G2", G2, "trained_por_G2.h5")],
            "Portuguese G3": [dnn(por, "G3", G3, "trained_por_G3.h5")]
        }
    )
    
    # format the accuracy scores to make them easier to read and more descriptive
    summary_table["Math G1"] = summary_table["Math G1"].map("{:.1f}%".format)
    summary_table["Math G2"] = summary_table["Math G2"].map("{:.1f}%".format)
    summary_table["Math G3"] = summary_table["Math G3"].map("{:.1f}%".format)
    summary_table["Portuguese G1"] = summary_table["Portuguese G1"].map("{:.1f}%".format)
    summary_table["Portuguese G2"] = summary_table["Portuguese G2"].map("{:.1f}%".format)
    summary_table["Portuguese G3"] = summary_table["Portuguese G3"].map("{:.1f}%".format)
    
    # change the index name it more clearly state that it is the accuracy scores being displayed
    summary_table = summary_table.rename(index={0: "Accuracy Score"})
    # show us the dataframe
    display(summary_table)
    
    return

In [6]:
target_summary()

,Math G1,Math G2,Math G3,Portuguese G1,Portuguese G2,Portuguese G3
Accuracy Score,65.7%,75.8%,81.8%,83.4%,83.4%,89.6%


# Conclusion
###### This actually surprised me how high the accuracy is on these models. I thought there would not be nearly enough rows for it to learn from. 
###### On the other hand it is less surprising that the models for the second and third trimesters are so good, because whether or not they passed the previous trimester is a really good predictor. 